In [12]:
pip install PyPDF2

DEPRECATION: pyodbc 4.0.0-unsupported has a non-standard version number. pip 24.0 will enforce this behaviour change. A possible replacement is to upgrade to a newer version of pyodbc or contact the author to suggest that they release a version with a conforming version number. Discussion can be found at https://github.com/pypa/pip/issues/12063
Note: you may need to restart the kernel to use updated packages.


In [13]:
import pandas as pd
import numpy as np
import nltk
import re
import random
import string
import warnings
warnings.simplefilter("ignore")
from PyPDF2 import PdfReader

In [14]:
pth = r"/home/justini/Documents/Projects/MEDIKEA PROJECTS/Medikea/Medical Assistant/STG _ NEMLIT 2021 (1)-1.pdf"

In [15]:
raw_doc1 = ''

In [16]:

pdf = PdfReader(pth)
for index, page in enumerate(pdf.pages):
    page = pdf.pages[index]
    raw_doc1 += page.extract_text()

In [17]:
with open('Dataset/full-doc.txt', 'w') as f:
    f.write(raw_doc1)
    f.close()

In [18]:
pg

NameError: name 'pg' is not defined

In [ ]:
def deseaese_checking(text):
    pattern = re.compile(r'^\d+\.\d+$')

    match = pattern.match(text)

    return bool(match)

def category_checking(text):
    pattern = re.compile(r'^\d+\.\d.\d+$')

    match = pattern.match(text)

    return bool(match)

: 

In [ ]:
dataset = {'heading': [], 'disease':[], 'categories':[], 'Clinical presentation':[], 'Investigations':[], 'Non-Pharmacological Treatment':[], 'Pharmacological Treatment':[]}

: 

In [ ]:
def saving_data_dictionary(heading, diesease, category, presentation, investigation, phTreatment, nonPhTreatment):
    if heading and diesease and category and presentation and investigation:
        dataset['heading'].append(heading)
        dataset['disease'].append(diesease)
        dataset['categories'].append(category)
        dataset['Clinical presentation'].append(presentation)
        dataset['Investigations'].append(investigation)
        dataset['Non-Pharmacological Treatment'].append(nonPhTreatment)
        dataset['Pharmacological Treatment'].append(phTreatment)


: 

In [ ]:
text = ''
chapter_text = ''
disease_text = ''
category_text = ''
Investigations_text = ''
clinical_presentation_text = ''
Pharmacological_text = ''
non_Pharmacological_text = ''
status_chapter = False
status_disease = False
category_status = False
clinical_presentation_status= False
Investigations_status = False
Pharmacological_status = False
non_Pharmacological_status = False
heading = ''
diesease =  ''
category = ''
investigation = ''
presentation = ''
phTreatment = ''
nonPhTreatment = ''

pdf = PdfReader(pth)
for index, page in enumerate(pdf.pages):
    if index >= 96:
        page = pdf.pages[index]
        raw_doc1 += page.extract_text()
        raw_doc2 = raw_doc1.replace('\t\t', '')
        
        for index, data in enumerate(raw_doc2.split(' ')):
            capitalized_word = data.istitle()
            extracted_alphabets = ''
            try:
                extracted_alphabets = re.sub("[^a-zA-Z]", "", data)
                if extracted_alphabets == 'CHAPTER':
                    status_chapter = True
                    text = 'current chapter'
                    chapter_text = ''
                    saving_data_dictionary(heading, diesease, category, presentation, investigation, phTreatment, nonPhTreatment)
                    
                    if Pharmacological_text !='' and Pharmacological_status == True:
                        Pharmacological_status =False
            except:
                pass
            
            if deseaese_checking(data) and text == 'current chapter':
                status_chapter = False
                status_disease = True
                disease_text = ''
                heading = chapter_text
                saving_data_dictionary(heading, diesease, category, presentation, investigation, phTreatment, nonPhTreatment)
                    
                if Pharmacological_text !='' and Pharmacological_status == True:
                    phTreatment = Pharmacological_text
                    Pharmacological_status =False
            
            if status_disease:
                phTreatment = Pharmacological_text
                nonPhTreatment = non_Pharmacological_text
                saving_data_dictionary(heading, diesease, category, presentation, investigation, phTreatment, nonPhTreatment)
                    
                if capitalized_word:
                    disease_text += data+ ' '
                else:
                    if disease_text !='':
                        diesease = disease_text
                        status_disease = False
               
            if category_checking(data):
                category_status = True
                category_text = ''
                
            if category_status:
                phTreatment = Pharmacological_text
                nonPhTreatment = non_Pharmacological_text
                saving_data_dictionary(heading, diesease, category, presentation, investigation, phTreatment, nonPhTreatment)
                    
                if capitalized_word:
                    category_text += data+ ' '
                else:
                    if category_text !='':
                        category_on = True
                        category = category_text
                        category_status = False
                        
            if extracted_alphabets =='Clinical':
                clinical_presentation_text = ''
                clinical_presentation_status = True
            
            if extracted_alphabets =='Investigations':
                clinical_presentation_status = False
                Investigations_text = ''
                Investigations_status =True
                presentation = clinical_presentation_text
                
            if status_chapter:
                chapter_text +=data +' '
            
            if clinical_presentation_status:
                clinical_presentation_text += data+ ' '
            
            if Investigations_status:
                try:
                    Investigations_text += data+ ' '
                except:
                    pass
                
            if non_Pharmacological_status:
                try:
                    non_Pharmacological_text += data+ ' '
                except:
                    pass
                
            if Pharmacological_status:
                try:
                    Pharmacological_text += data+ ' '
                except:
                    pass
                
            if extracted_alphabets == 'Pharmacological':
                Investigations_status = False
                Pharmacological_status = True
                Pharmacological_text = ''
                investigation = Investigations_text
                
            if extracted_alphabets == 'Non-Pharmacological':
                non_Pharmacological_text = ''
                non_Pharmacological_status =True

: 

In [ ]:
df = pd.DataFrame(dataset)

: 

In [ ]:
df.dropna(inplace=True)

: 

In [ ]:
df.drop_duplicates(inplace=True)

: 

In [ ]:
df

: 

In [ ]:
df.to_csv('more-Extracted-data.csv', index=False)

: 